## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf

In [157]:
tf.__version__

'1.15.0'

In [159]:
#Make sure tf 2.x is installed
!pip install -U tensorflow --quiet

False

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [94]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [95]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/content/drive/My Drive/Residency 6 Lab/prices.csv')

### Check all columns in the dataset

In [134]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date', 'symbol'], axis = 1, inplace = True) 

In [136]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [137]:
data1 = data.head(1000)
data1.shape

(1000, 5)

In [138]:
data1['volume'] = data1['volume'].div(1000000)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [139]:
data1.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

X = data1.copy()
y = data1['volume'].copy()

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size = 0.30,random_state=324)


In [141]:
X_train.shape[0] == y_train.shape[0]

True

In [142]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(700, 5)
(300, 5)
(700,)
(300,)


#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [144]:
from sklearn import preprocessing

X_train_scaled = preprocessing.scale(X_train)
X_train_scaled

array([[-3.02683250e-01, -3.11029505e-01, -3.02807150e-01,
        -3.03985463e-01, -1.52317386e-01],
       [-4.96933537e-01, -4.88134326e-01, -4.92534357e-01,
        -4.92576334e-01,  1.26143527e-01],
       [-8.73003563e-02, -7.53972951e-02, -8.58056193e-02,
        -7.25330483e-02, -1.29796540e-01],
       ...,
       [ 9.47999286e+00,  9.42922683e+00,  9.47402480e+00,
         9.42508312e+00,  5.01227637e-02],
       [-2.15358323e-02,  4.56127656e-05, -1.74493110e-02,
        -5.13111311e-03, -8.16190903e-02],
       [-6.80194941e-01, -6.85875988e-01, -6.79704569e-01,
        -6.80662908e-01,  3.50354973e-01]])

In [145]:
X_test_scaled = preprocessing.scale(X_test)
y_train_scaled = preprocessing.scale(y_train)
y_test_scaled = preprocessing.scale(y_test)

X_train_scaled = X_train_scaled.astype('float32')
X_test_scaled = X_test_scaled.astype('float32')
y_train_scaled = y_train_scaled.astype('float32')
y_test_scaled = y_test_scaled.astype('float32') 


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
w = tf.zeros(shape=(5,1)) 
b = tf.zeros(shape=(1)) 

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [182]:
for i in range(100):
    
    w, b = train(X_train_scaled, y_train_scaled, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, 
          loss(y_train_scaled, prediction(X_train_scaled, w, b)))

Current Loss on iteration 0 Tensor("Mean_609:0", shape=(), dtype=float32)
Current Loss on iteration 1 Tensor("Mean_611:0", shape=(), dtype=float32)
Current Loss on iteration 2 Tensor("Mean_613:0", shape=(), dtype=float32)
Current Loss on iteration 3 Tensor("Mean_615:0", shape=(), dtype=float32)
Current Loss on iteration 4 Tensor("Mean_617:0", shape=(), dtype=float32)
Current Loss on iteration 5 Tensor("Mean_619:0", shape=(), dtype=float32)
Current Loss on iteration 6 Tensor("Mean_621:0", shape=(), dtype=float32)
Current Loss on iteration 7 Tensor("Mean_623:0", shape=(), dtype=float32)
Current Loss on iteration 8 Tensor("Mean_625:0", shape=(), dtype=float32)
Current Loss on iteration 9 Tensor("Mean_627:0", shape=(), dtype=float32)
Current Loss on iteration 10 Tensor("Mean_629:0", shape=(), dtype=float32)
Current Loss on iteration 11 Tensor("Mean_631:0", shape=(), dtype=float32)
Current Loss on iteration 12 Tensor("Mean_633:0", shape=(), dtype=float32)
Current Loss on iteration 13 Tensor

In [119]:
X_train_scaled

array([[-3.0268326e-01, -3.1102949e-01, -3.0280715e-01, -3.0398548e-01,
        -1.5231739e-01],
       [-4.9693355e-01, -4.8813432e-01, -4.9253437e-01, -4.9257633e-01,
         1.2614353e-01],
       [-8.7300353e-02, -7.5397298e-02, -8.5805617e-02, -7.2533049e-02,
        -1.2979653e-01],
       ...,
       [ 9.4799929e+00,  9.4292269e+00,  9.4740248e+00,  9.4250832e+00,
         5.0122764e-02],
       [-2.1535832e-02,  4.5612767e-05, -1.7449312e-02, -5.1311129e-03,
        -8.1619091e-02],
       [-6.8019491e-01, -6.8587601e-01, -6.7970455e-01, -6.8066293e-01,
         3.5035497e-01]], dtype=float32)

### Get the shapes and values of W and b

In [80]:

print('Weights:\n', w)

Weights:
 Tensor("sub_401:0", shape=(5, 1), dtype=float32)


In [81]:
print('Bias:\n',b)

Bias:
 Tensor("sub_402:0", shape=(1,), dtype=float32)


### Model Prediction on 1st Examples in Test Dataset

In [164]:
result = prediction(X_test_scaled, w, b)
result

<tf.Tensor 'Add_612:0' shape=(300, 1) dtype=float32>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [194]:
iris_data = pd.read_csv('/content/drive/My Drive/Residency 6 Lab/Iris.csv')
iris_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [195]:
iris_data = pd.get_dummies( iris_data, prefix=['Species']) 
iris_data.drop(['Id'], axis = 1, inplace = True) 
iris_data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [200]:
target = iris_data[iris_data.columns[0:4]] 
feature = iris_data[iris_data.columns[4:7]] 
feature.head()
print(target.shape)
print(feature.shape)

(150, 4)
(150, 3)


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

#Add Dense Layer which provides 3 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, input_shape=(4,), activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [204]:

X1_train,X1_test,y1_train,y1_test= train_test_split(target,feature,test_size = 0.30,random_state=324)
print(X1_train.shape)
print(X1_test.shape)
print(y1_train.shape)
print(y1_test.shape)
print(target.shape)
print(feature.shape)

(105, 4)
(45, 4)
(105, 3)
(45, 3)
(150, 4)
(150, 3)


### Model Training 

In [0]:
model.fit(X1_train, y1_train, 
          validation_data=(X1_test, y1_test), 
          epochs=100,
          batch_size=X1_train.shape[0])

### Model Prediction

In [206]:
pred = model.predict(X1_test)
pred

ValueError: ignored

### Save the Model

In [0]:
model.save('mnist_lc_v2.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?